In [37]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [22]:
training_data = pd.read_csv('/content/drive/MyDrive/training.csv')
training_data.head()

,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,...,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
0,18453471,0.001578,0.999999,14.033335,0.681401,0.016039,0.451886,1.900433,1482.037476,0.066667,...,12290.760742,39264.398438,3.076006,4.003800,4.031514,458,-99,0,1866.300049,0.277559
1,5364094,0.000988,0.999705,5.536157,0.302341,0.142163,9.564503,0.865666,3050.720703,0.024022,...,16562.667969,7341.257812,3.228553,2.786543,2.975564,406,-99,0,1727.095947,0.225924
2,11130990,0.000877,0.999984,6.117302,0.276463,0.034746,1.970751,10.975849,3895.908691,0.055044,...,22695.388672,10225.309570,3.536903,2.865686,3.052810,196,-99,0,1898.588013,0.368630
3,15173787,0.000854,0.999903,5.228067,0.220739,0.076389,4.271331,3.276358,4010.781738,0.053779,...,16909.515625,9141.426758,3.087461,3.218034,2.375592,137,-99,0,1840.410034,0.246045
4,1102544,0.001129,0.999995,39.069534,1.898197,0.120936,4.984982,0.468348,4144.546875,0.004491,...,97612.804688,47118.785156,4.632295,4.711155,4.296878,477,-99,0,1899.793945,0.222060


In [17]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67553 entries, 0 to 67552
Data columns (total 51 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   67553 non-null  int64  
 1   LifeTime             67553 non-null  float64
 2   dira                 67553 non-null  float64
 3   FlightDistance       67553 non-null  float64
 4   FlightDistanceError  67553 non-null  float64
 5   IP                   67553 non-null  float64
 6   IPSig                67553 non-null  float64
 7   VertexChi2           67553 non-null  float64
 8   pt                   67553 non-null  float64
 9   DOCAone              67553 non-null  float64
 10  DOCAtwo              67553 non-null  float64
 11  DOCAthree            67553 non-null  float64
 12  IP_p0p2              67553 non-null  float64
 13  IP_p1p2              67553 non-null  float64
 14  isolationa           67553 non-null  int64  
 15  isolationb           67553 non-null 

In [19]:
training_data.isnull().sum()

id                     0
LifeTime               0
dira                   0
FlightDistance         0
FlightDistanceError    0
IP                     0
IPSig                  0
VertexChi2             0
pt                     0
DOCAone                0
DOCAtwo                0
DOCAthree              0
IP_p0p2                0
IP_p1p2                0
isolationa             0
isolationb             0
isolationc             0
isolationd             0
isolatione             0
isolationf             0
iso                    0
CDF1                   0
CDF2                   0
CDF3                   0
ISO_SumBDT             0
p0_IsoBDT              0
p1_IsoBDT              0
p2_IsoBDT              0
p0_track_Chi2Dof       0
p1_track_Chi2Dof       0
p2_track_Chi2Dof       0
p0_IP                  0
p1_IP                  0
p2_IP                  0
p0_IPSig               0
p1_IPSig               0
p2_IPSig               0
p0_pt                  0
p1_pt                  0
p2_pt                  0


In [23]:
training_data = training_data.drop(columns=['min_ANNmuon','production','mass', 'id'],axis=1)
training_data.head()

,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,...,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal
0,0.001578,0.999999,14.033335,0.681401,0.016039,0.451886,1.900433,1482.037476,0.066667,0.060602,...,448.369446,1393.246826,3842.096436,12290.760742,39264.398438,3.076006,4.003800,4.031514,458,0
1,0.000988,0.999705,5.536157,0.302341,0.142163,9.564503,0.865666,3050.720703,0.024022,0.019245,...,2033.918701,747.137024,8299.368164,16562.667969,7341.257812,3.228553,2.786543,2.975564,406,0
2,0.000877,0.999984,6.117302,0.276463,0.034746,1.970751,10.975849,3895.908691,0.055044,0.047947,...,2576.380615,963.652466,11323.134766,22695.388672,10225.309570,3.536903,2.865686,3.052810,196,0
3,0.000854,0.999903,5.228067,0.220739,0.076389,4.271331,3.276358,4010.781738,0.053779,0.006417,...,1351.734131,1685.003662,11502.081055,16909.515625,9141.426758,3.087461,3.218034,2.375592,137,0
4,0.001129,0.999995,39.069534,1.898197,0.120936,4.984982,0.468348,4144.546875,0.004491,0.037326,...,1755.792236,1282.428711,74117.117188,97612.804688,47118.785156,4.632295,4.711155,4.296878,477,0


In [24]:
y = training_data['signal']

In [25]:
training_data['DOCAone'] = np.tanh(training_data.DOCAone)

In [26]:
training_data.corr()

,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,...,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal
LifeTime,1.000000,0.138078,0.579831,0.022987,0.287670,0.200332,-0.066195,-0.143521,0.015162,0.023936,...,-0.108924,-0.090841,-0.027183,-0.032246,-0.030193,0.097809,0.076193,0.086031,-0.054257,0.097118
dira,0.138078,1.000000,0.173659,0.098117,-0.341075,-0.310822,-0.158035,0.165532,-0.139679,-0.126628,...,0.133825,0.092477,0.121596,0.142352,0.112078,0.033601,0.074935,0.069119,-0.094689,0.274832
FlightDistance,0.579831,0.173659,1.000000,0.555245,0.152521,0.109910,-0.055118,0.232469,-0.014266,0.000811,...,0.200722,0.144566,0.489946,0.567290,0.457292,0.408334,0.476077,0.434466,-0.020533,0.056076
FlightDistanceError,0.022987,0.098117,0.555245,1.000000,0.141275,0.005943,0.015808,0.027099,0.187208,0.199699,...,0.049803,0.012362,0.550069,0.637576,0.512659,0.658538,0.717577,0.688090,0.033219,-0.071107
IP,0.287670,-0.341075,0.152521,0.141275,1.000000,0.863057,0.184783,-0.208479,0.219216,0.223077,...,-0.157634,-0.122478,-0.030211,-0.020783,-0.008973,0.175058,0.172322,0.165635,0.150693,-0.470281
IPSig,0.200332,-0.310822,0.109910,0.005943,0.863057,1.000000,0.188232,-0.001465,0.087347,0.092583,...,0.004766,0.004116,-0.011676,0.002902,0.005145,0.015027,0.018356,0.009165,0.196747,-0.519312
VertexChi2,-0.066195,-0.158035,-0.055118,0.015808,0.184783,0.188232,1.000000,-0.048848,0.410172,0.401481,...,-0.041497,-0.015977,-0.021197,-0.024978,-0.004981,0.040217,0.030406,0.029615,0.147155,-0.328232
pt,-0.143521,0.165532,0.232469,0.027099,-0.208479,-0.001465,-0.048848,1.000000,-0.278499,-0.265123,...,0.814337,0.629898,0.364812,0.433180,0.338619,-0.261301,-0.209813,-0.229639,-0.017954,0.043643
DOCAone,0.015162,-0.139679,-0.014266,0.187208,0.219216,0.087347,0.410172,-0.278499,1.000000,0.188731,...,-0.240552,-0.079038,-0.102066,-0.056481,0.053314,0.297341,0.225610,0.185358,0.080383,-0.200780
DOCAtwo,0.023936,-0.126628,0.000811,0.199699,0.223077,0.092583,0.401481,-0.265123,0.188731,1.000000,...,-0.166837,-0.225767,0.002972,0.007039,-0.062343,0.193020,0.222511,0.270123,0.083170,-0.184562


In [27]:
X= training_data.drop(columns=['signal'])

In [28]:
sc = StandardScaler()
X_scaled= pd.DataFrame(sc.fit_transform(X), columns=X.columns)

In [29]:
x=X_scaled.values

In [30]:
y=y.values
y=y.reshape(-1,1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.33)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test,y_test)

0.8601803256627641

In [38]:
params = {"objective": "binary:logistic",
          "eta": 0.3,
          "max_depth": 5,
          "min_child_weight": 3,
          "silent": 1,
          "subsample": 0.7,
          "colsample_bytree": 0.7,
          "seed": 1}
xgb_model = xgb.XGBClassifier(objective="binary:logistic",params=params)
xgb_model.fit(X_train,y_train)

XGBClassifier(params={'colsample_bytree': 0.7, 'eta': 0.3, 'max_depth': 5,
                      'min_child_weight': 3, 'objective': 'binary:logistic',
                      'seed': 1, 'silent': 1, 'subsample': 0.7})

In [39]:
xgb_model.score(X_train,y_train)

0.8827662395050817